In [ ]:
from __future__ import division
import datetime
import time
import os
import sys

import numpy as np
from scipy import linalg
from matplotlib import rcParams
import matplotlib.pyplot as plt

sys.path.append('../')
from alg_tools_1d import dirac_recon_time, periodicSinc, distance

# various experiment settings
stop_cri = 'max_iter'  # stopping criteria: 1) mse; or 2) max_iter

In [ ]:
def plot_dirac(tk, ak, color='red', marker='*', ax=None, label=''):
    if ax is None:
        fig = plt.figure()
        ax = plt.gca()
    markerline211_1, stemlines211_1, baseline211_1 = \
        ax.stem(tk, ak, label=label)
    plt.setp(stemlines211_1, linewidth=1.5, color=color)
    plt.setp(markerline211_1, marker=marker, linewidth=1.5, markersize=8,
             markerfacecolor=color, mec=color)
    plt.setp(baseline211_1, linewidth=0)
    
def plot_diracs(ax1, tk, ak, tk_recon, ak_recon, title):
    c1 = [0, 0.447, 0.741]
    m1 ='^'
    plot_dirac(tk, ak, c1, m1, ax=ax1, label='Original Diracs')
    
    c2 = [0.850, 0.325, 0.098]
    m2 = '*'
    plot_dirac(tk_recon, ak_recon, c2, m2, ax=ax1, label='Estimated Diracs')

    plt.axhline(0, color='k')
    plt.xlim([0, tau])
    plt.ylim([1.17 * np.min(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis]))),
              1.17 * np.max(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis])))])
    plt.ylabel('amplitudes', fontsize=12)
    ax1.yaxis.set_label_coords(-0.095, 0.5)
    plt.legend(numpoints=1, loc=0, fontsize=9, framealpha=0.3,
               handletextpad=.2, columnspacing=0.6, labelspacing=0.05, ncol=2)
    plt.title(title, fontsize=12)


def plot_signal(ax2, t_plt, y_plt, t_samp, y_ell):
    line212_1 = ax2.plot(t_plt, y_plt, label='Ground Truth')
    plt.setp(line212_1, linestyle='-', color=[0, 0.447, 0.741], linewidth=1)

    line212_2 = ax2.plot(t_samp, y_ell, label='Samples')
    plt.setp(line212_2, marker='.', linestyle='None', markersize=5, color=[0.850, 0.325, 0.098])
    plt.ylim([1.05 * np.min(np.concatenate((y_plt, y_ell))),
              1.05 * np.max(np.concatenate((y_plt, y_ell)))])
    plt.ylabel(r'$x(t) * \mathrm{{sinc}}(B t)$', fontsize=12)
    plt.xlabel(r'$t$', fontsize=12)
    ax2.xaxis.set_label_coords(0.5, -0.21)
    ax2.yaxis.set_label_coords(-0.095, 0.5)
    plt.legend(numpoints=1, loc=0, fontsize=9, framealpha=0.3,
               handletextpad=.2, columnspacing=0.6, labelspacing=0.05, ncol=2)
    plt.show()

# Overview 

This notebook explains how to reconstruct a signal consisting of a $\tau$-periodic stream of Diracs at unknown locations. We will apply the following standard FRI-based workflow. 

## 1. Generate signal and measurements
1.We generate the FRI signal which we will then try reconstruct. We also simulate measurements by constructing a non-uniformly sampled, low-pass filtered signal, and add measurement noise.

<br><center>
$ x = \sum_{k' \in \mathbb{Z}} \sum_{k=1}^{K}  \alpha_k \delta(t - t_k - k' \tau ) $ (1) 
</center>

*CODE: Inspect the signal and make sure you understand its parameters.*

## 2. Find standard form
2.Since the signal it is FRI, we know that we can find a signal of the standard form with a 1-to-1 relation to the original signal:

<center>
$ \hat{x}_m = \sum_{k=1}^{K}(\beta_k * u_k^m) $ (2) 
</center>

*PEN AND PAPER: Find values of $\beta_k$ and $u_k$*. 

3.Since the above holds, we know that the signal can be anihilated by a filter $h$. 

*OPTIONAL: Show that for this simple example, this filter is given by*

$$ H(z) = c_0 \prod_{k=1}^K (1 - e^{-j\frac{2 \pi}{\tau} t_k} z^{-1}) $$

## 4. Find and implement $ G $ 
4.Once the signal is in form of 2, we need to identify how it is related to measurements y. 

*PEN AND PAPER: find the expression of matrix X such that $ G \hat{x} = y$*


## 5. Solve optimization
5.Now we have all the ingredients to solve the optimization of the form: 

<center>
find $ \hat{x}, h $ 
</center>

<center>
such that $ || y - G \hat{x} ||_2 \leq \epsilon $
</center>

<center>
and $ x * h = 0 $
</center>

*CODE: you do not have to implement this part, just inspect the obtained solution and make sure it is correct.*

## 6. Reconstruct original signal
6.Now that we have extracted the locations of the signal, what would you do to find the signal's amplitudes?  

## 1. Generate FRI signal (periodic stream of Diracs)

In [ ]:
np.random.seed(3)

K = 5  # number of Diracs
M = K # number of Fourier samples (at least K)
tau = 1  # period of the Dirac stream

# amplitudes of the Diracs
ak = np.sign(np.random.randn(K)) * (1 + (np.random.rand(K) - 0.5) / 1.)

# locations of the Diracs
############ TODO: does this have to be so complicated? 
#a = 4. / L
#uk = np.random.exponential(scale=1. / K, size=(K - 1, 1))
#tk = np.cumsum(a + (1. - K * a) * (1 - 0.1 * np.random.rand()) / uk.sum() * uk)
#tk = np.sort(np.hstack((np.random.rand() * tk[0] / 2., tk)) + (1 - tk[-1]) / 2.) * tau
tk = np.random.rand(K) * tau

# save Dirac parameter
time_stamp = datetime.datetime.now().strftime("%d-%m_%H_%M")
file_name = '../data/Dirac_Data_' + time_stamp + '.npz'
np.savez(file_name, tk=tk, ak=ak, K=K, time_stamp=time_stamp)
print('saved as', file_name)

############ TODO: plot the signal.
plot_dirac(tk, ak)

## Sample the FRI signal

In [ ]:
############ TODO: why do we not use much simpler time samples? 

def generate_time_samples(tau, M):
    # number of time domain samples
    L = (2 * M + 1)
    Tmax = tau / L  # the average sampling step size (had we used a uniform sampling setup)

    # generate the random sampling time instances
    t_samp = np.arange(0, L, dtype=float) * Tmax
    t_samp += np.sign(np.random.randn(L)) * np.random.rand(L) * Tmax / 2.
    # round t_samp to [0, tau)
    t_samp -= np.floor(t_samp / tau) * tau
    return t_samp

t_samp = generate_time_samples(tau, M)

############ TODO: Can we compute yl using the dirichlet kernel? 
def phi(t):
    ''' Dirichlet kernel evaluated at t.
    '''
    return np.sin(np.pi * B * t) / (B * tau * np.sin(np.pi * t / tau))


tl_grid, tk_grid = np.meshgrid(t_samp, tk)
yl = np.inner(ak, phi(tl_grid - tk_grid))

############ TODO: Plot the signal.

In [ ]:
def get_fourier_series_coefficients(ak, tk):
    ''' 
    Compute fourier series coefficients of periodic dirac stream.
    :param ak: vector of Dirac amplitudes
    :param tk: vector of Dirac locations

    :return: vector of Fourier series coefficients
    '''
    B = (2. * M + 1.) / tau  # bandwidth of the sampling filter
    ms = np.arange(-np.floor(B * tau / 2.), 1 + np.floor(B * tau / 2.))
    tk_grid, m_grid_gt = np.meshgrid(tk, ms)
    
    x_hat = 1. / tau * np.dot(np.exp(-2j * np.pi / tau * m_grid_gt * tk_grid), ak)
    return x_hat

In [ ]:
# load saved data
time_stamp = r'20-12_02_22'
stored_param = np.load(r'../data/Dirac_Data_' + time_stamp + r'.npz')
tk = stored_param['tk']
ak = stored_param['ak']

print(r'time stamp: ' + time_stamp +
      '\n=======================================\n')

def get_G():
    '''
    
    
    '''

x_hat_noiseless = get_fourier_series_coefficients(ak, tk)
# build the linear transformation matrix that links x_hat with the samples

m_grid, t_samp_grid = np.meshgrid(np.arange(-np.floor(B * tau / 2.), 1 + np.floor(B * tau / 2.)), t_samp)
G = 1. / B * np.exp(2j * np.pi / tau * m_grid * t_samp_grid)

In [ ]:
## generate noiseless signal
y_ell_noiseless = np.real(np.dot(G, x_hat_noiseless))

##### TODO verify that generated signal is equal to sampled signal.

assert np.isclose(y_ell_noiseless, y_ell).all()

In [ ]:
## generate noisy signal. 
P = float('inf')
L = (2 * M + 1)

noise = np.random.randn(L)
noise = noise / linalg.norm(noise) * linalg.norm(y_ell_noiseless) * 10 ** (-P / 20.)
y_ell = y_ell_noiseless + noise

## FRI reconstruction

In [ ]:
# noise energy, in the noiseless case 1e-10 is considered as 0
noise_level = np.max([1e-10, linalg.norm(noise)])
max_ini = 100  # maximum number of random initialisations
xhat_recon, min_error, c_opt, ini = dirac_recon_time(G, y_ell, K, noise_level, max_ini, stop_cri)

print(r'Noise level: {0:.2e}'.format(noise_level))
print(r'Minimum approximation error |a - Gb|_2: {0:.2e}'.format(min_error))

## Reconstruct signal

In [ ]:
z = np.roots(c_opt)
z = z / np.abs(z)
tk_recon = np.real(tau * 1j / (2 * np.pi) * np.log(z))
tk_recon = np.sort(tk_recon - np.floor(tk_recon / tau) * tau)


def get_ak(...):
    

# reconstruct amplitudes ak
Phi_recon = periodicSinc(np.pi * B * (np.reshape(t_samp, (-1, 1), order='F') -
                                      np.reshape(tk_recon, (1, -1), order='F')),
                         B * tau)
ak_recon = np.real(linalg.lstsq(Phi_recon, y_ell)[0])
# location estimation error
t_error = distance(tk_recon, tk)[0]

In [ ]:
fig = plt.figure(num=1, figsize=(5.5, 2.5), dpi=90)
ax1 = plt.axes([0.125, 0.59, 0.85, 0.31])

t_error_pow = np.int(np.floor(np.log10(t_error)))
title = 'reconstructed vs. original signal'
plot_diracs(ax1, tk, ak, tk_recon, ak_recon, title)


ax2 = plt.axes([0.125, 0.18, 0.85, 0.31])

t_plt = np.linspace(0, tau, num=np.max([10 * L, 1000]))
xs = np.arange(-np.floor(B * tau / 2.), 1 + np.floor(B * tau / 2.))
m_plt_grid, t_plt_grid = np.meshgrid(xs, t_plt)
G_plt = 1. / B * np.exp(2j * np.pi / tau * m_plt_grid * t_plt_grid)
y_plt = np.real(np.dot(G_plt, x_hat_noiseless))  # for plotting purposes only

plot_signal(ax2, t_plt, y_plt, t_samp, y_ell)

In [ ]:
K = 5  # number of Diracs
M = K * 8  # number of Fourier samples (at least K)
tau = 1  # period of the Dirac stream

# generate parameters for the periodic stream of Diracs
B = (2. * M + 1.) / tau  # bandwidth of the sampling filter
L = 2 * M + 1

t_samp = generate_time_samples(tau, M)

# compute the noiseless Fourier series coefficients
tk_grid, m_grid_gt = np.meshgrid(tk, np.arange(-np.floor(B * tau / 2.), 1 + np.floor(B * tau / 2.)))
x_hat_noiseless = 1. / tau * np.dot(np.exp(-2j * np.pi / tau * m_grid_gt * tk_grid), ak)

m_grid, t_samp_grid = np.meshgrid(np.arange(-np.floor(B * tau / 2.), 1 + np.floor(B * tau / 2.)), t_samp)

# build the linear transformation matrix that links x_hat with the samples
G = 1. / B * np.exp(2j * np.pi / tau * m_grid * t_samp_grid)
y_ell_noiseless = np.real(np.dot(G, x_hat_noiseless))

# add noise
P = 5
noise = np.random.randn(L)
noise = noise / linalg.norm(noise) * linalg.norm(y_ell_noiseless) * 10 ** (-P / 20.)
y_ell = y_ell_noiseless + noise

# noise energy, in the noiseless case 1e-10 is considered as 0
noise_level = np.max([1e-10, linalg.norm(noise)])
max_ini = 100  # maximum number of random initialisations

# FRI reconstruction
xhat_recon, min_error, c_opt, ini = dirac_recon_time(G, y_ell, K, noise_level, max_ini, stop_cri)

print(r'Noise level: {0:.2e}'.format(noise_level))
print(r'Minimum approximation error |a - Gb|_2: {0:.2e}'.format(min_error))

# reconstruct Diracs' locations tk
z = np.roots(c_opt)
z = z / np.abs(z)
tk_recon = np.real(tau * 1j / (2 * np.pi) * np.log(z))
tk_recon = np.sort(tk_recon - np.floor(tk_recon / tau) * tau)

# reconstruct amplitudes ak
Phi_recon = periodicSinc(np.pi * B * (np.reshape(t_samp, (-1, 1), order='F') -
                                      np.reshape(tk_recon, (1, -1), order='F')),
                         B * tau)
ak_recon = np.real(linalg.lstsq(Phi_recon, y_ell)[0])
# location estimation error
t_error = distance(tk_recon, tk)[0]
# plot reconstruction
plt.close()

In [ ]:
# plot reconstruction
fig = plt.figure(num=1, figsize=(5.5, 2.5), dpi=90)
# sub-figure 1
ax1 = plt.axes([0.125, 0.59, 0.85, 0.31])
markerline211_1, stemlines211_1, baseline211_1 = \
    ax1.stem(tk, ak, label='Original Diracs')
plt.setp(stemlines211_1, linewidth=1.5, color=[0, 0.447, 0.741])
plt.setp(markerline211_1, marker='^', linewidth=1.5, markersize=8,
         markerfacecolor=[0, 0.447, 0.741], mec=[0, 0.447, 0.741])
plt.setp(baseline211_1, linewidth=0)

markerline211_2, stemlines211_2, baseline211_2 = \
    plt.stem(tk_recon, ak_recon, label='Estimated Diracs')
plt.setp(stemlines211_2, linewidth=1.5, color=[0.850, 0.325, 0.098])
plt.setp(markerline211_2, marker='*', linewidth=1.5, markersize=10,
         markerfacecolor=[0.850, 0.325, 0.098], mec=[0.850, 0.325, 0.098])
plt.setp(baseline211_2, linewidth=0)

plt.axhline(0, color='k')
plt.xlim([0, tau])
plt.ylim([1.17 * np.min(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis]))),
          1.17 * np.max(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis])))])
# plt.xlabel(r'$t$', fontsize=12)
plt.ylabel('amplitudes', fontsize=12)
ax1.yaxis.set_label_coords(-0.095, 0.5)
plt.legend(numpoints=1, loc=0, fontsize=9, framealpha=0.3,
           handletextpad=.2, columnspacing=0.6, labelspacing=0.05, ncol=2)
t_error_pow = np.int(np.floor(np.log10(t_error)))
if np.isinf(P):
    plt.title(r'$K={0}$, $L={1}$, '
              r'$\mbox{{SNR}}=\mbox{{inf }}$dB, '
              r'$t_{{\mbox{{\footnotesize err}}}}={2:.2f}\times10^{other}$'.format(repr(K), repr(L),
                                                                                     t_error / 10 ** t_error_pow,
                                                                                     other='{' + str(
                                                                                         t_error_pow) + '}'),
              fontsize=12)
else:
    plt.title(r'$K={0}$, $L={1}$, '
              r'$\mbox{{SNR}}={2}$dB, '
              r'$t_{{\mbox{{\footnotesize err}}}}={3:.2f}\times10^{other}$'.format(repr(K), repr(L), repr(P),
                                                                                     t_error / 10 ** t_error_pow,
                                                                                     other='{' + str(
                                                                                         t_error_pow) + '}'),
              fontsize=12)

# sub-figure 2
t_plt = np.linspace(0, tau, num=np.max([10 * L, 1000]))
m_plt_grid, t_plt_grid = np.meshgrid(np.arange(-np.floor(B * tau / 2.),
                                               1 + np.floor(B * tau / 2.)),
                                     t_plt)
G_plt = 1. / B * np.exp(2j * np.pi / tau * m_plt_grid * t_plt_grid)
y_plt = np.real(np.dot(G_plt, x_hat_noiseless))  # for plotting purposes only

ax2 = plt.axes([0.125, 0.18, 0.85, 0.31])
line212_1 = ax2.plot(t_plt, y_plt, label='Ground Truth')
plt.setp(line212_1, linestyle='-', color=[0, 0.447, 0.741], linewidth=1)

line212_2 = ax2.plot(t_samp, y_ell, label='Samples')
plt.setp(line212_2, marker='.', linestyle='None', markersize=5, color=[0.850, 0.325, 0.098])
plt.ylim([1.05 * np.min(np.concatenate((y_plt, y_ell))),
          1.05 * np.max(np.concatenate((y_plt, y_ell)))])
plt.ylabel(r'$x(t) * \mathrm{{sinc}}(B t)$', fontsize=12)
plt.xlabel(r'$t$', fontsize=12)
ax2.xaxis.set_label_coords(0.5, -0.21)
ax2.yaxis.set_label_coords(-0.095, 0.5)
plt.legend(numpoints=1, loc=0, fontsize=9, framealpha=0.3,
           handletextpad=.2, columnspacing=0.6, labelspacing=0.05, ncol=2)
plt.show()